In [ ]:
import networkx as nx
import numpy as np
import random
import time
import matplotlib.pyplot as plt

def initialize_network(N, d, initial_defector_ratio):
    """
    Creates a d-regular graph and randomly assigns 'C' (cooperator) or 'D' (defector) states to nodes.
    """
    # Generate a random d-regular graph where each node has degree d
    G = nx.random_regular_graph(d, N)
    # Calculate the number of defectors based on the initial ratio
    num_defectors = int(N * initial_defector_ratio)
    # Create a list of states: 'D' for defectors, 'C' for cooperators
    states = ['D'] * num_defectors + ['C'] * (N - num_defectors)
    # Shuffle states to assign them randomly
    random.shuffle(states)
    # Assign states to nodes
    for node, state in zip(G.nodes(), states):
        G.nodes[node]['state'] = state
    return G

def update_strategy(G, alpha, u, discordant_edges):
    """
    Selects a discordant edge, calculates transition probability based on payoff difference,
    updates the state of one node, and refreshes the discordant edge set for the changed node.
    """
    if discordant_edges:
        # Convert discordant_edges set to tuple for random selection
        edge = random.choice(tuple(discordant_edges))
        # Identify cooperator (C) and defector (D) nodes in the edge
        c_node, d_node = edge if G.nodes[edge[0]]['state'] == 'C' else (edge[1], edge[0])
        # Calculate payoffs: cooperator gets 1 per cooperating neighbor; defector gets (1+u) per C, u per D
        payoff1 = sum(1 for n in G.neighbors(c_node) if G.nodes[n]['state'] == 'C')
        payoff2 = sum((1 + u) if G.nodes[n]['state'] == 'C' else u for n in G.neighbors(d_node))
        # Set transition probability to 0 if payoff difference exceeds 2, otherwise use logistic function
        probability_1_to_2 = 1 / (1 + np.exp(alpha * (payoff1 - payoff2))) if (payoff1 - payoff2) <= 2 else 0
        # Update state based on probability
        if random.random() < probability_1_to_2:
            G.nodes[c_node]['state'] = 'D'
            changed_node = c_node
        else:
            G.nodes[d_node]['state'] = 'C'
            changed_node = d_node
        # Remove the processed edge from discordant_edges
        discordant_edges.discard(edge)
        # Update discordant edge set for the changed node's neighbors
        for neighbor in G.neighbors(changed_node):
            potential_edge = tuple(sorted([changed_node, neighbor]))
            if G.nodes[changed_node]['state'] != G.nodes[neighbor]['state']:
                discordant_edges.add(potential_edge)
            else:
                discordant_edges.discard(potential_edge)
    return discordant_edges

def rewire(G, discordant_edges, strategy_type, p, threshold):
    """
    Selects a discordant edge, removes it, chooses a new connection target based on the strategy,
    adds the new edge, and updates the discordant edge set.
    """
    if discordant_edges:
        edge = random.choice(tuple(discordant_edges))
        c_node, d_node = edge if G.nodes[edge[0]]['state'] == 'C' else (edge[1], edge[0])
        G.remove_edge(*edge)
        discordant_edges.discard(edge)
        # Find nodes not currently connected to c_node (excluding itself and existing neighbors)
        neighbors_set = set(G.neighbors(c_node))
        potential_nodes = list(set(G.nodes()) - neighbors_set - {c_node})
        new_node = None

        # For hybrid strategy, choose sub-strategy based on cooperation fraction
        if strategy_type == 'hybrid':
            coop_fraction = calculate_cooperation_fraction(G)
            if coop_fraction < threshold:
                strategy_type = 'max_degree_C'
            else:
                strategy_type = 'min_degree_C'

        # Rewiring strategies
        if strategy_type == 'max_degree_C':
            max_degree = -1
            for node in potential_nodes:
                if G.nodes[node]['state'] == 'C' and G.degree[node] > max_degree:
                    max_degree = G.degree[node]
                    new_node = node
        elif strategy_type == 'min_degree_C':
            min_degree = float('inf')
            for node in potential_nodes:
                if G.nodes[node]['state'] == 'C' and G.degree[node] < min_degree:
                    min_degree = G.degree[node]
                    new_node = node
        elif strategy_type == 'probabilistic':
            if random.random() < p:
                cooperators = [node for node in potential_nodes if G.nodes[node]['state'] == 'C']
                new_node = random.choice(cooperators) if cooperators else random.choice(potential_nodes)
            else:
                new_node = random.choice(potential_nodes)
        elif strategy_type == 'random':
            if potential_nodes:
                new_node = random.choice(potential_nodes)

        # Fallback: if no specific node selected, choose randomly
        if new_node is None and potential_nodes:
            new_node = random.choice(potential_nodes)
        if new_node:
            G.add_edge(c_node, new_node)
            # Update discordant edges for c_node's new connections
            for neighbor in G.neighbors(c_node):
                potential_edge = tuple(sorted([c_node, neighbor]))
                if G.nodes[c_node]['state'] != G.nodes[neighbor]['state']:
                    discordant_edges.add(potential_edge)
                else:
                    discordant_edges.discard(potential_edge)
    return discordant_edges

def simulate(G, alpha, u, w, max_times, strategy_type, p, threshold):
    """
    Simulates network evolution until max iterations or no discordant edges remain (all edges homogeneous).
    Uses discordant_edges set to efficiently determine continuation condition.
    """
    iteration = 0
    # Initialize set of discordant edges (sorted tuples for undirected edges)
    discordant_edges = set(tuple(sorted((i, j))) for i, j in G.edges() if G.nodes[i]['state'] != G.nodes[j]['state'])
    while iteration < max_times and discordant_edges:
        if random.random() < w:
            discordant_edges = update_strategy(G, alpha, u, discordant_edges)
        else:
            discordant_edges = rewire(G, discordant_edges, strategy_type, p, threshold)
        iteration += 1
    return G

def calculate_cooperation_fraction(G):
    """
    Calculates the fraction of cooperating nodes in the network.
    """
    cooperators = sum(1 for node in G.nodes() if G.nodes[node]['state'] == 'C')
    return cooperators / G.number_of_nodes()

def simulate_and_average_cooperation(N, d, initial_defector_ratio, alpha, u, w, simulations, max_times, strategy_type, p, threshold):
    """
    Runs multiple simulations and returns the average cooperation fraction.
    """
    total_cooperation = 0
    for _ in range(simulations):
        G = initialize_network(N, d, initial_defector_ratio)
        simulate(G, alpha, u, w, max_times, strategy_type, p, threshold)
        total_cooperation += calculate_cooperation_fraction(G)
    return total_cooperation / simulations

def run_simulation_for_different_w(w_values, N, d, initial_defector_ratio, alpha, u_values, simulations, max_times, strategy_types, p, threshold):
    """
    Simulates across different w values, u values, and strategy types, returning results in a dictionary.
    """
    results = {}
    start_time = time.time()
    for strategy_type in strategy_types:
        for w in w_values:
            cooperation_fractions = []
            for u in u_values:
                avg_cooperation = simulate_and_average_cooperation(
                    N, d, initial_defector_ratio, alpha, u, w, simulations, max_times, strategy_type, p, threshold
                )
                cooperation_fractions.append(avg_cooperation)
            results[(strategy_type, w)] = cooperation_fractions
    print(f"Total time for all simulations: {time.time() - start_time:.2f} seconds")
    return results

def plot_multiple_cooperation_vs_u(u_values, results):
    """
    Plots the relationship between u (cost-to-benefit ratio) and the fraction of cooperators.
    """
    for (strategy_type, w), cooperation_fractions in results.items():
        plt.plot(u_values, cooperation_fractions, marker='o', label=f'{strategy_type}, w={w}')
    plt.xlabel('Cost-to-benefit ratio, u')
    plt.ylabel('Fraction of cooperators')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()


# -------------------------------
# Simulation example parameters
if __name__ == "__main__":
  N = 10000                   # Number of nodes
  d = 10                     # Degree of each node in the d-regular graph
  initial_defector_ratio = 0.5  # Initial fraction of defectors
  alpha = 30                  # Sensitivity parameter for strategy update
  u_values = np.linspace(0, 1, 51)  # Range of cost-to-benefit ratios
  w_values = [0.1]           # Probability weight for strategy update vs. rewiring
  simulations = 1             # Number of simulation runs per condition
  max_times = 10**12       # Maximum iterations per simulation
  strategy_types = ['min_degree_C']  # Rewiring strategy to test
  p = 1                       # Probability for probabilistic rewiring
  threshold = 0.5             # Threshold for hybrid strategy

  # Run simulations and plot results
  results = run_simulation_for_different_w(w_values, N, d, initial_defector_ratio, alpha, u_values, simulations, max_times, strategy_types, p, threshold)
  plot_multiple_cooperation_vs_u(u_values, results)